In this notebook, I will use Stan to try to fit a really simply fixed effects model to the first 40 answers for each Toppr user. We assume that each Toppr user i has a fixed probability of answering each of the first 20 questions correctly $\alpha_{i,0}$ and a fixed probability of answering the second 20 questions correctly $\alpha_{i,1}$.  Let $Y_{it}$ be an indicator for whether student i gets the t question correct and d[t] a function which equals 1 if t is less than or equal to 10 and 2 if t is greater than 10.  Then the model we fit is...

$$ P(Y_{it}=1) = \alpha_{i,d[t]} $$

$$ \alpha_d \sim N(\mu_d,\sigma^2_d) $$

$$ \mu_d \sim N(.5,.3); \sigma \sim U(0,1) $$

Our goal is to compare $\sigma_1$ vs $\sigma_2$. 